## zk-ml: truly private machine learning on blockchain

Peiyuan Liao, Milo Cress, ludens

### **Thesis: modern machine learning platforms lack security and transperancy**

* https://www.kaggle.com/, https://codalab.org/, https://www.crowdai.org/
* Experiences of competing in Kaggle as a Competitions Grandmaster (Peiyuan):
 - user downloads dataset
 - trains model
 - uploads it (???)
 - evaluated against some test set (???) 
 - and some metric (???)
 - some random meetings with the organizers (???)
 - prize is delivered 
* https://www.kaggle.com/c/deepfake-detection-challenge/discussion/157983#885598
 - Top ML competition team denied of 500k USD due to opaque data issues
* https://www.kaggle.com/c/global-wheat-detection/discussion/167298
 - Several under-the-hood re-runs, edits to private test set, arbitrarily invalidating submissions
 
### Solution: zk-SNARKS

* ML models usually operates in float, double and half precisions, but zk-SNARKS work in prime fields
* Quantization!
  - Prime complement for negative numbers
  - Slight precision loss on both sides (sometimes better, sometimes worse)
  - Division circuits are very expernsive but doable (negatives, quotioent-remainder relationships)
  - Multiply-add approximations for nonlinearities (sigmoid, tanh, etc.)

### The zk-ml protocol

* What zk-SNARKs provide: 
 - succint proof of computation that neural networks perform certain way under certain datasets
* What ETH provides:
 - securely post and claim bounties
 - reliable way to transfer model for key agreement protocol
 
* The circuit: Linear Regression
    - Checks that all the public inputs that defines a ml dataset (x,y pairs with quantization constants) hashes correctly to hash_input, as this will be the main key for bounty deposit and claim
    - confirms that the actual model params correctly encrypts to the public model params. This way, the bounty issuer can simply use the public key and the encrypted params in the calldata along with their private key to restore the params
    - Performs the model inference and confirms that it indeed has a certain measure wrt public targets.
* The contract
    - Organizers post bounties with IPFS links to datasets
    - Competitors query for bounty and download datasets
    - Competitors trains model, quantizes it and generate proofs locally
    - (if conditions are met) Competitors uploads proof and claims bounty
    - (if conditions are not met)Organizers

### Future works

* More ML models
 - Language models: I-BERT (https://arxiv.org/abs/2101.01321)
 - Gradient Boosted Decision Trees, XGBoost, LightGBM, etc: trees are representable as circuits
 - Compiler from neural network IR to groth16 verifier (zokrates w. optimizations)
* Protocol revamp
 - Multiple contributions to a contract: proxy contracts
 - Pushing beyond contract size limit:
     - Diamond patterns, libraries
     - 16/8/4-bit quantization and bit packing inside uint256
     - Moving zk-SNARK computation to cryptoprocessors
 - Preventing frontrunners: bounty issuer can see the calldata, front-run the transcation, and remove the bounty
 - Preventing over-fitting for over-parameterized models (MLP, ConvNets): two stage competition -> public leaderboard proof and private leaderboard proofs
 - Preventing adversarial organizers creating invalid test sets (random noise, exceptionally hard, etc).
     - User public keys are kept private during public leaderboard
     - Slashing if organizer does not relesase private dataset in-time
     - Slashing if private dataset behaves in an adversarial manner 
* DAO
 

In [52]:
alias zkml source ~/.nvm/nvm.sh >/dev/null && nvm use 14.0.0 >/dev/null && yarn >/dev/null && ./zkml

In [53]:
alias prepare source ~/.nvm/nvm.sh >/dev/null && nvm use 14.0.0 >/dev/null && yarn >/dev/null && yarn prepare-demo > /dev/null

In [38]:
prepare

In [71]:
zkml list_datasets

Available datasets:
[
  '14797455496207951391356508759149962584765968173479481191220882411966396840571'
]


In [72]:
zkml list_bounties --hash '14797455496207951391356508759149962584765968173479481191220882411966396840571'

Available bounties on dataset: 14797455496207951391356508759149962584765968173479481191220882411966396840571
[
  {
    PubKey1: '8385930133524036559273104048419821016491875134467519785353152799140128167921',
    PubKey2: '17155255312018438018746349179262607517493307326242830912102091150670145263900',
    MSEcap: '12888',
    Bounty: '49.0',
    Issuer: '0x1CBd3b2770909D4e10f157cABC84C7264073C9Ec',
    IPFS: 'QmWLRJVL5uViT7h64bdeUM3GKMWP9DSWRggGC8igDuQdHR'
  }
]


Command:

```
zkml download_dataset --hash '14797455496207951391356508759149962584765968173479481191220882411966396840571' --publickey ./keys/out_public.json --walletprivatekey ./keys/.private_key --mse 18406 --path ./ipfs_dataset
```

In [73]:
zkml download_dataset --hash '14797455496207951391356508759149962584765968173479481191220882411966396840571' --publickey ./keys/out_public.json --walletprivatekey ./keys/.private_key --mse 12888 --path ./ipfs_dataset

In [74]:
!ls ./ipfs_dataset

X.npy Y.npy


In [75]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import LinearRegression

In [76]:
X = np.load('./ipfs_dataset/X.npy')
Y = np.load('./ipfs_dataset/Y.npy')

In [77]:
Y.shape, X.shape

((20, 1), (20, 4))

In [78]:
# Instantiating LinearRegression() Model
lr = LinearRegression()

# Training/Fitting the Model
lr.fit(X, Y)

# Making Predictions
pred = lr.predict(X)

# Evaluating Model's Performance
print('Mean Squared Error:', mean_squared_error(y_test, pred))

mse = mean_squared_error(y_test, pred)

Mean Squared Error: 0.07865899873764468


In [79]:
W = lr.coef_.reshape(-1, 1)
b = lr.intercept_.reshape(-1, 1)
X = X_test.values[:]
Yt_expected = y_test.values[:].reshape(-1, 1)

np.save('model/W.npy',W)
np.save('model/b.npy',b)
np.save('dataset/X.npy',X)
np.save('dataset/Y.npy',Yt_expected)

In [80]:
shuffle = lambda x: np.random.shuffle(x)
shuffle(W)
shuffle(b)
print(mean_squared_error(y_test, np.matmul(X_test, W.reshape(-1)) + b.reshape(-1)))

np.save('model_shuffled/W.npy',W)
np.save('model_shuffled/b.npy',b)

0.07941453957968247


In [81]:
from scripts import *
from copy import deepcopy
import json

data = dict(
    alpha_X = 0,
    beta_X = 8,

    alpha_W = -1,
    beta_W = 8,

    alpha_Y = 0,
    beta_Y = 8,

    alpha_Yt = 0,
    beta_Yt = 8,

    alpha_b = 0,
    beta_b = 8,

    alpha_R = -1,
    beta_R = 1,

    alpha_S = 0,
    beta_S = 0.4,

    m = 20,
    p = 4,
    n = 1,
    
    mse_target = 0.07864
)

json.dump(data, open('./settings.json', 'w'), indent = 2)

Command:

```
zkml claim_bounty --payment 0x2546BcD3c84621e976D8185a91A922aE77ECEc30 --model ./model_shuffled --dataset ./dataset --publickey ./keys/out_public.json --settings ./settings.json
```

In [82]:
zkml claim_bounty --payment 0x2546BcD3c84621e976D8185a91A922aE77ECEc30 --model ./model_shuffled --dataset ./dataset --publickey ./keys/out_public.json --settings ./settings.json

Mean Squared Error actual:  0.07941453957968254
... quantized  13014
Mean Squared Error simulated:  0.07939327351522920684
... quantized  13011
Circuit Outputs:
13011
Proof took 17.02 s
ERROR: Invalid proof
An unexpected error occurred:

Error: Could not verify the proof
    at SimpleTaskDefinition.action (/Users/liaopeiyuan/Documents/GitHub/zkml-core/eth/hardhat.config.js:281:26)
    at Environment._runTaskDefinition (/Users/liaopeiyuan/Documents/GitHub/zkml-core/eth/node_modules/hardhat/src/internal/core/runtime-environment.ts:217:14)
    at Environment.run (/Users/liaopeiyuan/Documents/GitHub/zkml-core/eth/node_modules/hardhat/src/internal/core/runtime-environment.ts:129:14)
    at main (/Users/liaopeiyuan/Documents/GitHub/zkml-core/eth/node_modules/hardhat/src/internal/cli/cli.ts:197:5)


Command:

```
zkml claim_bounty --payment 0x2546BcD3c84621e976D8185a91A922aE77ECEc30 --model ./model  --dataset ./dataset  --publickey ./keys/out_public.json --settings ./settings.json
```

In [83]:
zkml claim_bounty --payment 0x2546BcD3c84621e976D8185a91A922aE77ECEc30 --model ./model  --dataset ./dataset  --publickey ./keys/out_public.json --settings ./settings.json

Mean Squared Error actual:  0.0786589987376446
... quantized  12891
Mean Squared Error simulated:  0.07864249526948667579
... quantized  12888
Circuit Outputs:
12888
Proof took 17.131 s
INFO: OK!
Paying 0x2546BcD3c84621e976D8185a91A922aE77ECEc30
With balance
10850.6
Your Public Key: 
[
  1700643759997457569245863490312877944508569018329107232774239121212875587628n,
  12432032059012451124939976221853806268790884588923275757468717217243035579610n
]
Your Private Key: 
16643980665868843823629143933455494516978640287615526294342242121812095812996n
Current Balance
10899.6


In [84]:
zkml list_bounties --hash 14797455496207951391356508759149962584765968173479481191220882411966396840571

Available bounties on dataset: 14797455496207951391356508759149962584765968173479481191220882411966396840571
[]


In [85]:
zkml list_datasets

Available datasets:
[]
